# DATA CLEARNING FOR NPP

In [1]:
%matplotlib inline

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

## Tool functions preparation

### Combine base and knowledge data

In [2]:
def combined_base_knowledge_data(base_data, knowledge_data):
    combined_base_knowledge_data = base_data.set_index('ID').join(knowledge_data.set_index('ID'))
    combined_base_knowledge_data = combined_base_knowledge_data.fillna(combined_base_knowledge_data.median())
    return combined_base_knowledge_data

### Function definition for year_fill_na

In [3]:
# fill null year 
def year_fill_na(oragin_data, years = [2015, 2016, 2017]):
    # pick null year data
    null_years = oragin_data.loc[oragin_data.year.isna()]
    
    # fill year 
    for year in years:
        IDs = oragin_data[["ID"]].loc[oragin_data.year == year]
        for null_year_id in null_years["ID"].unique():
            tmp = IDs.loc[IDs.ID == null_year_id]
            if tmp.empty:
                index = oragin_data.loc[(oragin_data.ID == null_year_id) 
                                        & (oragin_data.year.isna())].index.tolist()
                if len(index) != 0:
                    oragin_data.loc[index[0]:index[0], "year"] = year 
                
    new_data = oragin_data.fillna(oragin_data.median())
    return new_data;

### Function definition for combined_new_year_money_data

In [4]:
def combined_new_year_money_data(new_year_data, new_money_data):
    return pd.merge(new_year_data, new_money_data, on=['ID', 'year'])

### Split year_money_data

In [1]:
def split_data(combined_new_year_money_data, years=[2015, 2016, 2017]):
    pre = combined_new_splited_year_money_data = None
    is_first = True
    
    # split data with year
    for year in years:
        combined_new_year_data = combined_new_year_money_data.loc[
            combined_new_year_money_data.year == year].set_index('ID').add_suffix("_%d"%(year)).drop(columns=['year_%d'%(year)])
        
        if is_first:
            pre = combined_new_year_data
            is_first = False
        else:
            # 如果是17年的数据，则只合并净利润
            if year == 2017:
                combined_new_year_data = combined_new_year_data[['净利润_2017']]
            # marge data with ID
            combined_new_splited_year_money_data = pd.merge(pre, 
                                                    combined_new_year_data, 
                                                    on=['ID'])
            pre = combined_new_splited_year_money_data


    return combined_new_splited_year_money_data

## Training Data Set

### Load original training data

In [6]:
import pandas as pd

In [7]:
# load training data
base_train_data = pd.read_csv("data/train/base-train.csv")
year_train_data = pd.read_csv("data/train/year-train.csv")
knowledge_train_data = pd.read_csv("data/train/knowledge-train.csv")
money_train_data = pd.read_csv("data/train/money-train.csv")

### base and knowledge data

In [8]:
base_train_data.head()

,ID,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例
0,5981446,2008.0,1190.0,服务业,湖南,农民专业合作社,自然人,0.91
1,5993858,2008.0,6670.0,零售业,湖南,集体所有制企业,企业法人,0.91
2,5990025,2008.0,9100.0,社区服务,湖北,农民专业合作社,NaN,0.51
3,5982670,2002.0,5870.0,商业服务业,广东,股份有限公司,企业法人,0.63
4,5990751,2005.0,9850.0,工业,江西,集体所有制企业,企业法人,0.58


In [9]:
base_train_data.describe()

,ID,注册时间,注册资本,控制人持股比例
count,1.731900e+04,17131.000000,17140.000000,17135.000000
mean,5.989033e+06,2007.040336,5044.745041,0.754200
std,6.350533e+03,4.328399,2852.898559,0.144789
min,5.978034e+06,2000.000000,100.000000,0.510000
25%,5.983530e+06,2003.000000,2580.000000,0.630000
50%,5.989066e+06,2007.000000,5040.000000,0.750000
75%,5.994520e+06,2011.000000,7490.000000,0.880000
max,6.000000e+06,2014.000000,10000.000000,1.000000


In [10]:
knowledge_train_data.head()

,ID,专利,商标,著作权
0,5978034,0.0,1.0,1.0
1,5978036,0.0,0.0,1.0
2,5978037,1.0,1.0,1.0
3,5978038,0.0,0.0,0.0
4,5978039,0.0,0.0,0.0


In [11]:
knowledge_train_data.describe()

,ID,专利,商标,著作权
count,1.731900e+04,17142.000000,17143.000000,17136.000000
mean,5.989033e+06,0.342667,0.362772,0.377684
std,6.350533e+03,0.474615,0.480814,0.484822
min,5.978034e+06,0.000000,0.000000,0.000000
25%,5.983530e+06,0.000000,0.000000,0.000000
50%,5.989066e+06,0.000000,0.000000,0.000000
75%,5.994520e+06,1.000000,1.000000,1.000000
max,6.000000e+06,1.000000,1.000000,1.000000


#### combine base and knowledge training data

In [12]:
combined_base_knowledge_train_data = combined_base_knowledge_data(base_train_data, knowledge_train_data)
combined_base_knowledge_train_data.head()

,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例,专利,商标,著作权
ID,,,,,,,,,,
5981446,2008.0,1190.0,服务业,湖南,农民专业合作社,自然人,0.91,1.0,0.0,0.0
5993858,2008.0,6670.0,零售业,湖南,集体所有制企业,企业法人,0.91,0.0,0.0,0.0
5990025,2008.0,9100.0,社区服务,湖北,农民专业合作社,NaN,0.51,1.0,0.0,1.0
5982670,2002.0,5870.0,商业服务业,广东,股份有限公司,企业法人,0.63,1.0,0.0,1.0
5990751,2005.0,9850.0,工业,江西,集体所有制企业,企业法人,0.58,0.0,0.0,0.0


### year and money data

In [13]:
year_train_data.head(10)

,ID,year,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,净利润,纳税总额,所有者权益合计
0,5978034,2015.0,868.0,67200.0,61600.0,181440.0,127008.0,18144.0,72576.0,36288.0,5600.0
1,5978036,2015.0,556.0,71640.0,127360.0,358200.0,179100.0,71640.0,143280.0,143280.0,-55720.0
2,5978037,2015.0,426.0,19770.0,13180.0,39540.0,15816.0,19770.0,7908.0,23724.0,6590.0
3,5978038,2015.0,815.0,101700.0,96050.0,203400.0,81360.0,20340.0,0.0,40680.0,5650.0
4,5978039,2015.0,450.0,107100.0,202300.0,385560.0,308448.0,154224.0,38556.0,231336.0,-95200.0
5,5978040,2015.0,825.0,22920.0,25785.0,57300.0,40110.0,11460.0,17190.0,11460.0,-2865.0
6,5978041,2015.0,838.0,70000.0,99750.0,287000.0,114800.0,28700.0,86100.0,114800.0,-29750.0
7,5978042,2015.0,823.0,311520.0,457545.0,498432.0,249216.0,99686.4,99686.4,99686.4,-146025.0
8,5978043,2015.0,877.0,86760.0,39765.0,399096.0,159638.4,79819.2,39909.6,239457.6,46995.0
9,5978044,2015.0,958.0,29900.0,53820.0,107640.0,86112.0,43056.0,10764.0,64584.0,-23920.0


In [14]:
year_train_data.describe()

,ID,year,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,净利润,纳税总额,所有者权益合计
count,5.195400e+04,51458.000000,51454.000000,51408.000000,5.147700e+04,5.144100e+04,5.146400e+04,5.144600e+04,5.140600e+04,5.143100e+04,51419.000000
mean,5.989033e+06,2016.000039,510.511292,135813.864185,1.636242e+05,3.451787e+05,2.073263e+05,1.031948e+05,5.207380e+04,1.160021e+05,-27982.231568
std,6.350048e+03,0.816370,283.046739,135822.244472,1.969579e+05,4.413745e+05,2.775580e+05,1.537658e+05,1.164659e+05,1.898123e+05,108949.735773
min,5.978034e+06,2015.000000,20.000000,100.000000,0.000000e+00,1.100000e+02,4.960000e+01,1.240000e+01,-4.190500e+05,0.000000e+00,-800120.000000
25%,5.983530e+06,2015.000000,267.000000,36315.000000,3.307500e+04,5.874000e+04,3.359850e+04,1.411200e+04,0.000000e+00,6.656000e+03,-54000.000000
50%,5.989065e+06,2016.000000,510.000000,89420.000000,9.219000e+04,1.795680e+05,1.038700e+05,4.584640e+04,1.200770e+04,4.362400e+04,150.000000
75%,5.994519e+06,2017.000000,755.000000,191100.000000,2.156000e+05,4.515160e+05,2.656644e+05,1.240872e+05,6.383080e+04,1.395360e+05,8780.000000
max,5.999999e+06,2017.000000,1000.000000,838100.000000,1.619520e+06,4.190500e+06,2.873600e+06,1.729800e+06,1.436800e+06,2.264976e+06,415380.000000


In [15]:
money_train_data.head()

,ID,year,债权融资额度,债权融资成本,股权融资额度,股权融资成本,内部融资和贸易融资额度,内部融资和贸易融资成本,项目融资和政策融资额度,项目融资和政策融资成本
0,5978034,2015.0,0.0,0.0,0.0,0.000,54432.0,3265.92,0.0,0.000
1,5978036,2015.0,0.0,0.0,14328.0,573.120,0.0,0.00,0.0,0.000
2,5978037,2015.0,0.0,0.0,0.0,0.000,0.0,0.00,790.8,47.448
3,5978038,2015.0,10170.0,813.6,0.0,0.000,0.0,0.00,0.0,0.000
4,5978039,2015.0,0.0,0.0,30844.8,1233.792,0.0,0.00,0.0,NaN


In [16]:
money_train_data.describe()

,ID,year,债权融资额度,债权融资成本,股权融资额度,股权融资成本,内部融资和贸易融资额度,内部融资和贸易融资成本,项目融资和政策融资额度,项目融资和政策融资成本
count,5.195400e+04,51429.000000,51474.000000,51433.000000,51429.000000,51424.000000,5.147300e+04,51452.000000,51415.000000,51428.000000
mean,5.989033e+06,2016.000233,3330.930800,267.087725,5201.204070,208.387604,2.639850e+04,1582.824998,1020.993718,61.371247
std,6.350048e+03,0.816504,8830.859453,707.145117,18152.186526,727.102121,8.113653e+04,4868.402364,3007.802466,180.649317
min,5.978034e+06,2015.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,5.983530e+06,2015.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
50%,5.989065e+06,2016.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
75%,5.994519e+06,2017.000000,0.000000,0.000000,0.000000,0.000000,6.948000e+02,39.339000,22.100000,1.561500
max,5.999999e+06,2017.000000,82110.000000,6568.800000,303922.500000,12156.900000,1.257150e+06,75429.000000,37240.000000,2234.400000


#### year data fill na

In [17]:
# fill null
new_year_train_data = year_fill_na(year_train_data)
new_year_train_data.set_index(['ID', 'year'])

new_year_train_data.head()

,ID,year,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,净利润,纳税总额,所有者权益合计
0,5978034,2015.0,868.0,67200.0,61600.0,181440.0,127008.0,18144.0,72576.0,36288.0,5600.0
1,5978036,2015.0,556.0,71640.0,127360.0,358200.0,179100.0,71640.0,143280.0,143280.0,-55720.0
2,5978037,2015.0,426.0,19770.0,13180.0,39540.0,15816.0,19770.0,7908.0,23724.0,6590.0
3,5978038,2015.0,815.0,101700.0,96050.0,203400.0,81360.0,20340.0,0.0,40680.0,5650.0
4,5978039,2015.0,450.0,107100.0,202300.0,385560.0,308448.0,154224.0,38556.0,231336.0,-95200.0


#### money data fill na

In [18]:
# fill null
new_money_train_data = year_fill_na(money_train_data)
new_money_train_data.set_index(['ID', 'year'])

# new_money_train_data.head(8)
new_money_train_data.head()

,ID,year,债权融资额度,债权融资成本,股权融资额度,股权融资成本,内部融资和贸易融资额度,内部融资和贸易融资成本,项目融资和政策融资额度,项目融资和政策融资成本
0,5978034,2015.0,0.0,0.0,0.0,0.000,54432.0,3265.92,0.0,0.000
1,5978036,2015.0,0.0,0.0,14328.0,573.120,0.0,0.00,0.0,0.000
2,5978037,2015.0,0.0,0.0,0.0,0.000,0.0,0.00,790.8,47.448
3,5978038,2015.0,10170.0,813.6,0.0,0.000,0.0,0.00,0.0,0.000
4,5978039,2015.0,0.0,0.0,30844.8,1233.792,0.0,0.00,0.0,0.000


#### Merge new year and money data

In [19]:
# Merge new year and money data
combined_new_year_money_train_data = combined_new_year_money_data(new_year_train_data, new_money_train_data)
combined_new_year_money_train_data.head()

,ID,year,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,净利润,纳税总额,所有者权益合计,债权融资额度,债权融资成本,股权融资额度,股权融资成本,内部融资和贸易融资额度,内部融资和贸易融资成本,项目融资和政策融资额度,项目融资和政策融资成本
0,5978034,2015.0,868.0,67200.0,61600.0,181440.0,127008.0,18144.0,72576.0,36288.0,5600.0,0.0,0.0,0.0,0.000,54432.0,3265.92,0.0,0.000
1,5978036,2015.0,556.0,71640.0,127360.0,358200.0,179100.0,71640.0,143280.0,143280.0,-55720.0,0.0,0.0,14328.0,573.120,0.0,0.00,0.0,0.000
2,5978037,2015.0,426.0,19770.0,13180.0,39540.0,15816.0,19770.0,7908.0,23724.0,6590.0,0.0,0.0,0.0,0.000,0.0,0.00,790.8,47.448
3,5978038,2015.0,815.0,101700.0,96050.0,203400.0,81360.0,20340.0,0.0,40680.0,5650.0,10170.0,813.6,0.0,0.000,0.0,0.00,0.0,0.000
4,5978039,2015.0,450.0,107100.0,202300.0,385560.0,308448.0,154224.0,38556.0,231336.0,-95200.0,0.0,0.0,30844.8,1233.792,0.0,0.00,0.0,0.000


### merge splited data

In [20]:
splited_year_money_train_data = split_data(combined_new_year_money_train_data)
splited_year_money_train_data.head()

,从业人数_2015,资产总额_2015,负债总额_2015,营业总收入_2015,主营业务收入_2015,利润总额_2015,净利润_2015,纳税总额_2015,所有者权益合计_2015,债权融资额度_2015,...,所有者权益合计_2016,债权融资额度_2016,债权融资成本_2016,股权融资额度_2016,股权融资成本_2016,内部融资和贸易融资额度_2016,内部融资和贸易融资成本_2016,项目融资和政策融资额度_2016,项目融资和政策融资成本_2016,净利润_2017
ID,,,,,,,,,,,,,,,,,,,,,
5978034,868.0,67200.0,61600.0,181440.0,127008.0,18144.0,72576.0,36288.0,5600.0,0.0,...,-11200.0,3920.0,313.6,0.0,0.0,0.0,0.00,0.0,0.00,69888.0
5978036,556.0,71640.0,127360.0,358200.0,179100.0,71640.0,143280.0,143280.0,-55720.0,0.0,...,150.0,0.0,0.0,0.0,0.0,19104.0,1146.24,0.0,0.00,0.0
5978037,426.0,19770.0,13180.0,39540.0,15816.0,19770.0,7908.0,23724.0,6590.0,0.0,...,-65900.0,0.0,0.0,0.0,0.0,0.0,0.00,3954.0,237.24,7249.0
5978038,815.0,101700.0,96050.0,203400.0,81360.0,20340.0,0.0,40680.0,5650.0,10170.0,...,31075.0,5650.0,452.0,0.0,0.0,0.0,0.00,0.0,0.00,94468.0
5978039,450.0,107100.0,202300.0,385560.0,308448.0,154224.0,38556.0,231336.0,-95200.0,0.0,...,5950.0,0.0,0.0,0.0,0.0,0.0,0.00,1249.5,74.97,5712.0


## combine all data to train

In [21]:
train = pd.merge(combined_base_knowledge_train_data, 
                 splited_year_money_train_data, 
                 on=['ID'])

In [22]:
train.head()

,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例,专利,商标,著作权,...,所有者权益合计_2016,债权融资额度_2016,债权融资成本_2016,股权融资额度_2016,股权融资成本_2016,内部融资和贸易融资额度_2016,内部融资和贸易融资成本_2016,项目融资和政策融资额度_2016,项目融资和政策融资成本_2016,净利润_2017
ID,,,,,,,,,,,,,,,,,,,,,
5981446,2008.0,1190.0,服务业,湖南,农民专业合作社,自然人,0.91,1.0,0.0,0.0,...,-22015.0,4760.0,380.8,0.0,0.000,0.0,0.000,0.0,0.00,8353.8
5993858,2008.0,6670.0,零售业,湖南,集体所有制企业,企业法人,0.91,0.0,0.0,0.0,...,-43355.0,0.0,0.0,0.0,0.000,22411.2,1344.672,0.0,0.00,-31215.6
5990025,2008.0,9100.0,社区服务,湖北,农民专业合作社,NaN,0.51,1.0,0.0,1.0,...,9100.0,0.0,0.0,0.0,0.000,0.0,0.000,5824.0,349.44,22932.0
5982670,2002.0,5870.0,商业服务业,广东,股份有限公司,企业法人,0.63,1.0,0.0,1.0,...,-196645.0,0.0,0.0,23010.4,920.416,0.0,0.000,0.0,0.00,162012.0
5990751,2005.0,9850.0,工业,江西,集体所有制企业,企业法人,0.58,0.0,0.0,0.0,...,-147750.0,0.0,0.0,0.0,0.000,0.0,0.000,13002.0,780.12,302592.0


## Save train

In [23]:
train.to_csv("data/train/train.csv")

In [24]:
train.describe()

,注册时间,注册资本,控制人持股比例,专利,商标,著作权,从业人数_2015,资产总额_2015,负债总额_2015,营业总收入_2015,...,所有者权益合计_2016,债权融资额度_2016,债权融资成本_2016,股权融资额度_2016,股权融资成本_2016,内部融资和贸易融资额度_2016,内部融资和贸易融资成本_2016,项目融资和政策融资额度_2016,项目融资和政策融资成本_2016,净利润_2017
count,17318.000000,17318.000000,17318.000000,17318.000000,17318.000000,17318.000000,17318.000000,17318.000000,1.731800e+04,1.731800e+04,...,17318.000000,17318.000000,17318.000000,17318.000000,17318.000000,1.731800e+04,17318.000000,17318.000000,17318.000000,1.731800e+04
mean,2007.040305,5044.437579,0.754168,0.339185,0.359106,0.373715,511.037244,119960.741425,1.446556e+05,3.023359e+05,...,-27023.729068,3275.980656,262.736168,5248.856576,209.614746,2.673025e+04,1601.213209,982.977503,58.983292,5.807233e+04
std,4.304634,2837.994603,0.144013,0.473446,0.479752,0.483803,282.487380,124668.893716,1.823982e+05,3.970797e+05,...,107513.451234,8735.387142,699.176265,18523.398198,740.012428,8.108578e+04,4856.527677,2902.001776,173.664989,1.280454e+05
min,2000.000000,100.000000,0.510000,0.000000,0.000000,0.000000,20.000000,100.000000,0.000000e+00,1.100000e+02,...,-778440.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,-4.190500e+05
25%,2003.000000,2600.000000,0.630000,0.000000,0.000000,0.000000,269.000000,28712.500000,2.542875e+04,4.788000e+04,...,-51640.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00
50%,2007.000000,5040.000000,0.750000,0.000000,0.000000,0.000000,510.000000,77105.000000,7.757750e+04,1.511730e+05,...,150.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,1.400400e+04
75%,2011.000000,7460.000000,0.880000,1.000000,1.000000,1.000000,755.000000,169372.500000,1.890900e+05,3.924285e+05,...,8640.000000,0.000000,0.000000,0.000000,0.000000,7.029000e+02,42.511500,0.000000,0.000000,7.210620e+04
max,2014.000000,10000.000000,1.000000,1.000000,1.000000,1.000000,1000.000000,743250.000000,1.410440e+06,3.482700e+06,...,392850.000000,79840.000000,6387.200000,303922.500000,12156.900000,1.135200e+06,68112.000000,37240.000000,2234.400000,1.436800e+06


## Test Data Set

### Load original testing data

In [25]:
# load testing data
base_test_data = pd.read_csv("data/test/base-test.csv")
year_test_data = pd.read_csv("data/test/year-test.csv")
knowledge_test_data = pd.read_csv("data/test/knowledge-test.csv")
money_test_data = pd.read_csv("data/test/money-test.csv")

### base and knowledge data

In [26]:
# base_test_data.head()
# base_test_data.describe()
# knowledge_test_data.head()

#### combine base and knowledge training data

In [27]:
combined_base_knowledge_test_data = combined_base_knowledge_data(base_test_data, knowledge_test_data)
combined_base_knowledge_test_data.describe()

,注册时间,注册资本,控制人持股比例,专利,商标,著作权
count,4068.000000,4068.00000,4068.000000,4068.000000,4068.000000,4068.000000
mean,2007.014749,5018.40708,0.756534,0.352753,0.370452,0.382006
std,4.354105,2835.10610,0.145807,0.477885,0.482985,0.485938
min,2000.000000,100.00000,0.510000,0.000000,0.000000,0.000000
25%,2003.000000,2570.00000,0.630000,0.000000,0.000000,0.000000
50%,2007.000000,5005.00000,0.760000,0.000000,0.000000,0.000000
75%,2011.000000,7440.00000,0.880000,1.000000,1.000000,1.000000
max,2014.000000,10000.00000,1.000000,1.000000,1.000000,1.000000


### year and money data

In [28]:
year_test_data.describe()
money_test_data.describe()

,ID,year,债权融资额度,债权融资成本,股权融资额度,股权融资成本,内部融资和贸易融资额度,内部融资和贸易融资成本,项目融资和政策融资额度,项目融资和政策融资成本
count,8.136000e+03,8136.000000,8050.000000,8052.000000,8048.000000,8050.000000,8.041000e+03,8063.000000,8066.000000,8049.000000
mean,5.988991e+06,2015.500000,3278.145963,262.655221,4596.200728,184.874702,2.389549e+04,1421.144389,954.212125,57.290170
std,6.292912e+03,0.500031,8701.237782,696.672434,16395.617808,662.100357,7.381415e+04,4408.561432,2833.793732,170.423267
min,5.978045e+06,2015.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,5.983605e+06,2015.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
50%,5.988886e+06,2015.500000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
75%,5.994502e+06,2016.000000,270.000000,22.080000,0.000000,0.000000,4.356000e+02,13.464000,0.000000,0.000000
max,5.999996e+06,2016.000000,73425.000000,5874.000000,231601.500000,9264.060000,1.008000e+06,60480.000000,33145.000000,1988.700000


#### year data fill na

In [29]:
# fill null
new_year_test_data = year_fill_na(year_test_data, years=[2015, 2016])
new_year_test_data.set_index(['ID', 'year'])

new_year_test_data.describe()

print("unique ID count in base data:", base_test_data["ID"].nunique())
print("2015 unique ID count in year data:", new_year_test_data["ID"].loc[new_year_test_data.year==2015].nunique())
print("2016 unique ID count in year data:", new_year_test_data["ID"].loc[new_year_test_data.year==2016].nunique())

unique ID count in base data: 4068
2015 unique ID count in year data: 4068
2016 unique ID count in year data: 4068


#### money data fill na

In [30]:
# fill null
new_money_test_data = year_fill_na(money_test_data, years=[2015, 2016])
new_money_test_data.set_index(['ID', 'year'])

# new_money_train_data.head(8)
new_money_test_data.describe()

,ID,year,债权融资额度,债权融资成本,股权融资额度,股权融资成本,内部融资和贸易融资额度,内部融资和贸易融资成本,项目融资和政策融资额度,项目融资和政策融资成本
count,8.136000e+03,8136.000000,8136.000000,8136.000000,8136.000000,8136.000000,8.136000e+03,8136.000000,8136.000000,8136.000000
mean,5.988991e+06,2015.500000,3243.494961,259.943441,4546.487643,182.920520,2.361647e+04,1408.393217,946.002335,56.677554
std,6.292912e+03,0.500031,8661.613631,693.574687,16313.626906,658.862686,7.342677e+04,4390.782357,2822.951385,169.611918
min,5.978045e+06,2015.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,5.983605e+06,2015.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
50%,5.988886e+06,2015.500000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
75%,5.994502e+06,2016.000000,139.000000,13.160000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
max,5.999996e+06,2016.000000,73425.000000,5874.000000,231601.500000,9264.060000,1.008000e+06,60480.000000,33145.000000,1988.700000


In [31]:
print("unique ID count in base data:", base_test_data["ID"].nunique())
print("2015 unique ID count in money data:", new_money_test_data["ID"].loc[new_money_test_data.year==2015].nunique())
print("2016 unique ID count in money data:", new_money_test_data["ID"].loc[new_money_test_data.year==2016].nunique())

unique ID count in base data: 4068
2015 unique ID count in money data: 4068
2016 unique ID count in money data: 4068


#### Merge new year and money data

In [32]:
combined_new_year_money_test_data = combined_new_year_money_data(new_year_test_data, new_money_test_data)
combined_new_year_money_test_data.head()

,ID,year,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,净利润,纳税总额,所有者权益合计,债权融资额度,债权融资成本,股权融资额度,股权融资成本,内部融资和贸易融资额度,内部融资和贸易融资成本,项目融资和政策融资额度,项目融资和政策融资成本
0,5978045,2015,485.0,72000.0,140400.0,136800.0,54720.0,54720.0,-13680.0,0.0,-68400.0,0.0,0.0,0.0,0.000,0.0,0.000,720.0,43.20
1,5978049,2015,715.0,23820.0,9925.0,97662.0,68363.4,9766.2,9766.2,58597.2,13895.0,0.0,0.0,0.0,0.000,29298.6,1757.916,0.0,0.00
2,5978057,2015,841.0,45250.0,36200.0,149325.0,119460.0,44797.5,0.0,29865.0,9050.0,0.0,0.0,0.0,0.000,0.0,0.000,2262.5,135.75
3,5978060,2015,32.0,214800.0,411700.0,837720.0,502632.0,83772.0,335088.0,502632.0,-196900.0,21480.0,1718.4,0.0,0.000,0.0,0.000,0.0,0.00
4,5978072,2015,447.0,8700.0,7250.0,9570.0,7656.0,3828.0,3828.0,5742.0,1450.0,0.0,0.0,765.6,30.624,0.0,0.000,0.0,0.00


In [33]:
print("unique ID count in combined data:", combined_new_year_money_test_data["ID"].nunique())

unique ID count in combined data: 4068


### merge splited data

In [34]:
splited_year_money_test_data = split_data(combined_new_year_money_test_data, years=[2015, 2016])
splited_year_money_test_data.head()

,从业人数_2015,资产总额_2015,负债总额_2015,营业总收入_2015,主营业务收入_2015,利润总额_2015,净利润_2015,纳税总额_2015,所有者权益合计_2015,债权融资额度_2015,...,纳税总额_2016,所有者权益合计_2016,债权融资额度_2016,债权融资成本_2016,股权融资额度_2016,股权融资成本_2016,内部融资和贸易融资额度_2016,内部融资和贸易融资成本_2016,项目融资和政策融资额度_2016,项目融资和政策融资成本_2016
ID,,,,,,,,,,,,,,,,,,,,,
5978045,485.0,72000.0,140400.0,136800.0,54720.0,54720.0,-13680.0,0.0,-68400.0,0.0,...,0.0,1800.0,0.0,0.0,4950.0,198.0,0.0,0.0,0.0,0.000
5978049,715.0,23820.0,9925.0,97662.0,68363.4,9766.2,9766.2,58597.2,13895.0,0.0,...,18103.2,-39700.0,0.0,0.0,0.0,0.0,0.0,0.0,952.8,57.168
5978057,841.0,45250.0,36200.0,149325.0,119460.0,44797.5,0.0,29865.0,9050.0,0.0,...,139008.0,9050.0,0.0,0.0,0.0,0.0,0.0,0.0,2896.0,173.760
5978060,32.0,214800.0,411700.0,837720.0,502632.0,83772.0,335088.0,502632.0,-196900.0,21480.0,...,330792.0,-80550.0,0.0,0.0,0.0,0.0,0.0,0.0,3759.0,225.540
5978072,447.0,8700.0,7250.0,9570.0,7656.0,3828.0,3828.0,5742.0,1450.0,0.0,...,3393.0,2900.0,435.0,34.8,0.0,0.0,0.0,0.0,0.0,0.000


### merge all data to test

In [35]:
test = pd.merge(combined_base_knowledge_test_data, 
                 splited_year_money_test_data, 
                 on=['ID'])

In [36]:
test.head()

,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人持股比例,专利,商标,著作权,...,纳税总额_2016,所有者权益合计_2016,债权融资额度_2016,债权融资成本_2016,股权融资额度_2016,股权融资成本_2016,内部融资和贸易融资额度_2016,内部融资和贸易融资成本_2016,项目融资和政策融资额度_2016,项目融资和政策融资成本_2016
ID,,,,,,,,,,,,,,,,,,,,,
5992473,2010.0,8670.0,交通运输业,广东,有限责任公司,企业法人,0.54,1.0,1.0,1.0,...,4161.6,-34680.0,0.0,0.0,1248.48,49.9392,0.0,0.00,0.0,0.0
5999375,2010.0,3400.0,服务业,广东,集体所有制企业,企业法人,0.90,0.0,1.0,1.0,...,0.0,-25500.0,0.0,0.0,0.00,0.0000,14688.0,881.28,0.0,0.0
5981121,2014.0,3350.0,商业服务业,福建,有限责任公司,企业法人,0.51,1.0,1.0,0.0,...,49580.0,-26800.0,3350.0,268.0,0.00,0.0000,0.0,0.00,0.0,0.0
5978928,2012.0,7690.0,工业,广西,股份有限公司,企业法人,0.81,0.0,0.0,0.0,...,0.0,-46140.0,0.0,0.0,18456.00,738.2400,0.0,0.00,0.0,0.0
5979981,2013.0,4850.0,交通运输业,广西,股份有限公司,企业法人,0.61,1.0,0.0,1.0,...,113490.0,38800.0,0.0,0.0,15132.00,605.2800,0.0,0.00,0.0,0.0


### save testing data set

In [37]:
test.to_csv("data/test/test.csv")